In [1]:
from grid_game import Game
import numpy as np
import random


In [124]:
def print_policy():
    for i in range(5):
        print("",end="| ")
        for j in range(5):
            if (i,j) in Q_table:
                best_action=sorted(Q_table[(i,j)].keys(),key=lambda x:Q_table[(i,j)][x],reverse=True)[0]
                print(best_action,end=" | ")
            else:
                print("X",end=" | ")
        print()
        print("---------------------")

In [125]:
Q_table={}
update_ct_table={}
n_iteration=10000
iteration=0
gamma=0.9
epsilon_0=0.97
epsilon_decay=0.0001
learning_rate_0=0.5

#purely random action with epsilon greedy (off policy)
Q_change=1
tolerance=0.001
for iteration in range(n_iteration):
    Game.reset_game()
    Q_change=0
    steps=0
    while not Game.gameover:
        if Game.state not in Q_table:
            Game.next_step_random()
        else:
            epsilon=epsilon_0/(1+epsilon_decay*update_ct_table[Game.state])
            if random.random()<epsilon:
                Game.next_step_random()
            else:
                best_action=sorted(Q_table[Game.state].keys(),key=lambda x:Q_table[Game.state][x],reverse=True)[0]
                Game.next_step(best_action)
        steps+=1
        if steps>1000:
            break
    #update Q-table
    for s in Game.state_record:
        if s not in Q_table:
            Q_table[s]={"U":0,"D":0,"L":0,"R":0}
            update_ct_table[s]=0
    index=0
    max_index=len(Game.state_record)-1
    for s,a,r in zip(Game.state_record,Game.action_record,Game.reward_record):
        if index==max_index:
            next_Qsa_max=0
        else:
            next_Qsa_max=max(Q_table[Game.state_record[index+1]].values())
        update_ct_table[s]+=1
        learning_rate=learning_rate_0/(1+epsilon_decay*update_ct_table[s])
        Q_old=Q_table[s][a]
        Q_table[s][a]=(1-learning_rate)*Q_table[s][a]+\
        learning_rate*(r+gamma*next_Qsa_max)
        Q_change+=abs(Q_old-Q_table[s][a])
        index+=1
    if Q_change<tolerance:
        break
print("total iterations:",iteration)
print_policy()    


total iterations: 123
| R | R | D | L | L | 
---------------------
| D | X | D | X | X | 
---------------------
| R | R | R | R | X | 
---------------------
| U | X | X | U | U | 
---------------------
| U | R | R | U | U | 
---------------------


In [88]:
print(Q_table[(3,3)])

{'U': 0.5130978657817403, 'D': -1.0967877568914237, 'L': -3.8475102139086474, 'R': 0.7428397936750647}


In [108]:
print(update_ct_table)

{(4, 0): 478, (3, 0): 336, (2, 0): 252, (1, 0): 251, (0, 0): 167, (0, 1): 85, (0, 2): 82, (2, 1): 176, (2, 2): 92, (4, 1): 274, (4, 2): 129, (4, 3): 96, (3, 3): 45, (3, 4): 46, (1, 2): 94, (2, 3): 50, (0, 3): 68, (4, 4): 92, (0, 4): 82}
